In [1]:
import sys
import re
import requests
import json
from bs4 import BeautifulSoup

import urllib2
import socket

In [2]:
def load_credentials():
    lines = [line.rstrip('\n') for line in open('credentials.ini')]
    chars_to_strip = " \'\""
    for line in lines:
        if "client_id" in line:
            client_id = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]
        if "client_secret" in line:
            client_secret = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]
        #Currently only need access token to run, the other two perhaps for future implementation
        if "client_access_token" in line:
            client_access_token = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]

    return client_id, client_secret, client_access_token

In [3]:
# Genius API credentials (available globally)
client_id, client_secret, client_access_token = load_credentials()
genius_url = "http://api.genius.com"
headers = {'Authorization': 'Bearer ' + client_access_token}

In [4]:
def search_genius(search_term):             
    querystring = "http://api.genius.com/search?q=" + urllib2.quote(search_term) + "&page=" + str(1)        
    # querystring = "http://api.genius.com/" + urllib2.quote(search_term) + "&page=" + str(1)       
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + client_access_token)   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break    

    return json.loads(raw)

In [5]:
def get_song_and_artist_ids(song_title, artist_name):  
    json_obj = search_genius(song_title + " " + artist_name)
    body = json_obj["response"]["hits"]
    body = body[0] # Just keep the first hit (for now)
    song_api   = body['result']['api_path']
    artist_api = body['result']['primary_artist']['api_path']
    
    return song_api, artist_api

**Download all song lyrics for a given artist**

In [174]:
# It's dumb, but you have to use an artist's song to get their artist ID
artist_name ='Kanye West'
song_title = 'Champion' # I should make this automated by getting first song from artist search
song_id, artist_id = get_song_and_artist_ids(song_title,artist_name)
id_num = int(artist_id.split('/')[-1])

# Okay, we have the artist API id, let's get a list of all of their songs on Genius
all_song_ids = []
# for page in range(1,10):
page = 1
while True:
    request = urllib2.Request(genius_url + artist_id + '/songs' + '?page=%d' % page)
    request.add_header("Authorization", "Bearer " + client_access_token)
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)")
    response = urllib2.urlopen(request, timeout=4)
    raw = response.read()
    json_obj = json.loads(raw)      
    songs = json_obj['response']['songs']
    
    # Keep track of song API paths if the primary artist is correct
    [all_song_ids.append(song['api_path']) for song in songs if song['primary_artist']['id'] == id_num]            

    num_songs = len(songs)    
    if json_obj['response']['next_page']==None:
        if page==1 & num_songs == 0:
            print("No results for: " + search_term)
        break      
    print("Page {0} -- all songs {1}:".format(page, num_songs))    
    page += 1
    
print('Total songs found: {0}'.format(len(all_song_ids)))

Page 1 -- all songs 20:
Page 2 -- all songs 19:
Page 3 -- all songs 19:
Page 4 -- all songs 19:
Page 5 -- all songs 20:
Page 6 -- all songs 20:
Page 7 -- all songs 18:
Page 8 -- all songs 20:
Page 9 -- all songs 19:
Page 10 -- all songs 20:
Page 11 -- all songs 20:
Page 12 -- all songs 20:
Page 13 -- all songs 20:
Page 14 -- all songs 20:
Page 15 -- all songs 19:
Page 16 -- all songs 19:
Page 17 -- all songs 20:
Page 18 -- all songs 20:
Page 19 -- all songs 20:
Page 20 -- all songs 20:
Page 21 -- all songs 20:
Page 22 -- all songs 20:
Page 23 -- all songs 20:
Page 24 -- all songs 20:
Page 25 -- all songs 20:
Page 26 -- all songs 19:
Page 27 -- all songs 19:
Page 28 -- all songs 20:
Page 29 -- all songs 20:
Page 30 -- all songs 20:
Page 31 -- all songs 20:
Page 32 -- all songs 20:
Page 33 -- all songs 20:
Page 34 -- all songs 20:
Page 35 -- all songs 19:
Page 36 -- all songs 18:
Page 37 -- all songs 19:
Page 38 -- all songs 20:
Page 39 -- all songs 20:
Page 40 -- all songs 18:
Page 41 -

In [175]:
def lyrics_from_song_api_path(song_api_path):
    # Use BeautifulSoup to scrape lyrics off of a Genius song URL
    querystring = genius_url + song_api_path      
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + client_access_token)   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break

    json_obj = json.loads(raw)  

    # Get the URL to the song lyrics
    path = json_obj['response']['song']['path']
    page_url = "http://genius.com" + path
    page = requests.get(page_url)
    html = BeautifulSoup(page.text, "html.parser")    
    [h.extract() for h in html('script')]        
    lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore').decode('ascii')
    lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
    lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses
    return str(lyrics)

In [176]:
# This is annoying, Genius doesn't seem to differentiate between interviews and actual songs
# It just calls them all songs. You'd think there'd be a distinction in the metadata.
lyrics = lyrics_from_song_api_path(all_song_ids[4])
print(lyrics)


I told her to drive over after dude crib
Bring a couple new friends
Them last girls was too thick
How you expect me to hit?
You wanna watch Clueless
I wanna watch Foolish
Excuse me, miss, I don't appreciate your rudeness
But if you strip, I will appreciate your nudeness
I heard the best head come from girls that is toothless
Yeah my game ruthless, yeah my car ruthless
Spit that "Through The Wire" over Chaka Khan, ruthless
She said she had a friend look like Janet Jackson
That bitch came over lookin' more like Freddie Jackson
Nah, nah, nah, baby, won't be no time for no action
I'm just gon' play this "Slow Jamz," holla at your man
Old school like Melle Mel
Or we could go to the D where Drelle Drel
Or DC, PG, or ATL
Back in Chi, I mean my life like a magazine
Honey wanna know the details about my ebony
Well that's XXL
Father penthouse, yeah that's upscale
Mademoiselle got a hell of a black tail
I'm feelin' the vibe, her body and soul
I heard you stay in a metropolitan home
Well let's ki

In [177]:
def write_lyrics_to_file(lyrics,artist=''):   
    if artist!='':
        filename = 'Lyrics_{0}.txt'.format(artist.replace(' ',''))
    else:
        filename = "Lyrics.txt"
    with open(filename, "a") as text_file:
        text_file.write('\n' + lyrics)

In [178]:
# Output text file containing the lyrics of every identified song
for i in range(len(all_song_ids)):
    lyrics = lyrics_from_song_api_path(all_song_ids[i])
    write_lyrics_to_file(lyrics,artist_name)

**Get song lyrics by searching song and artist name**

In [161]:
song_title = 'yesterday'
artist_name = 'the beatles'
song_api_path, artist_api_path = get_song_and_artist_ids(song_title, artist_name)   
lyrics = lyrics_from_song_api_path(song_api_path)   
print(lyrics)


Yesterday
All my troubles seemed so far away
Now it looks as though they're here to stay
Oh, I believe in yesterday
Suddenly
I'm not half the man I used to be
There's a shadow hanging over me
Oh, yesterday came suddenly
Why she had to go
I don't know, she wouldn't say
I said something wrong
Now I long for yesterday
Yesterday
Love was such an easy game to play
Now I need a place to hide away
Oh, I believe in yesterday
Why she had to go
I don't know, she wouldn't say
I said something wrong
Now I long for yesterday
Yesterday
Love was such an easy game to play
Now I need a place to hide away
Oh, I believe in yesterday
